In [1]:
%load_ext autoreload
%autoreload 2

import jCMIP as jc
import os

In [2]:
# historical:
cmips = ['5','6']
EXPs  = ['historical']
styr  = 1970
fnyr  = 1999

In [4]:
# ScenarioMIP:
cmips = ['5','6']
EXPs  = ['rcp26','rcp45','rcp85','ssp126','ssp245','ssp585']
styr  = 2070
fnyr  = 2099

In [ ]:
vars  = ['tas']
vtype = 'Amon'

# Find all models with valid data make make slurm file:
for cmip in cmips:
    Clist = jc.readList(('../CMIP' + cmip + 'list'))
    Models = list(Clist.keys())
    for mm in Models:
        Model = Clist[mm]
        print(Model.name)
        if ((Model.Ogrid != 'Unstructured')):                
            for var in vars:
                for EXP in EXPs:
                    ENSs = Model.getENSs(var=var,EXP=EXP,vtype=vtype,gtype='*')
                    for ENS in ENSs:
                        gtype = 'gn'
                        # Check if files needed for computation exist:
                        Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)
                        if ((len(Files) == 0) & (cmip == '6')):
                            gtype = 'gr'
                            Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)
                            if ((len(Files) == 0) & (cmip == '6')):
                                gtype = 'gr1'
                                Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)

                        if len(Files) != 0:
                            # Check if computation has already been completed:
                            #outfile = ('/home/users/jmecking001/DATADIR/ACSIS/CMIP6/SeaLevel/' + var + '_' 
                            #           + Model.name + '_' + EXP + '_' + ENS + '_' + gtype + '_' 
                            #           + str(styr) + '01-' + str(fnyr) + '12_SC.nc')
                            
                            outfile = (Model.savedir + Model.name + '/' + EXP + '/' + ENS + '/' + var + '_' + Model.name + '_' + EXP + '_'
                                           + ENS + '_' + gtype + '_' + str(styr) + '01-' + str(fnyr) + '12_SC.nc')
                            
                            if not os.path.isfile(outfile):
                                Files.sort()
                                stf = int(Files[0].split('_')[-1].split('-')[0][:4])
                                fnf = int(Files[-1].split('_')[-1].split('-')[1][:4])
                                if ((stf <= styr) & (fnf >= fnyr)):
                                    print('computing: ' + outfile)
                                    # Generate submit script for lotus:
                                    lfile = ('slurm_files/SC_mean_' + var + '_' + Model.name + '_' +  EXP + '_' 
                                             + ENS + '_' + str(styr) + '01-' + str(fnyr) + '12_.slurm')

                                    line1 = '#!/bin/bash \n'
                                    line2 = '#SBATCH --account=short4hr \n'
                                    line3 = '#SBATCH -p short-serial-4hr \n'
                                    line4 = '#SBATCH -o %J.out \n'
                                    line5 = '#SBATCH -e %J.err \n'
                                    line6 = '#SBATCH --time 4:00:00 \n'
                                    line7 = '\n'
                                    line8 = 'conda activate working \n'
                                    line9 = ('python ../compute_means_SC.py ' + cmip + ' ' + Model.name + ' ' + EXP + ' ' 
                                             + ENS + ' ' + var + ' ' + vtype + ' ' + str(styr) + ' ' + str(fnyr) + ' ' 
                                             + outfile + ' ' + gtype + ' ../' + lfile)

                                    # Out of memory models (second pass):
                                    #line2 = '#SBATCH -p high-mem \n'
                                    #line3 = '#SBATCH --mem=32000 \n'

                                    file = open(lfile,'w') 
                                    file.writelines([line1, line2, line3, line4, line5, line6, line7, line8, line9])

                                    file.close()

In [5]:
vars  = ['tos','thetao','uo','vo','so','thkcello','volcello','tauuo','tauvo','zos','msftbarot','hfds','wfo','hfy']
#vars  = ['hfy']
vtype = 'Omon'

# Find all models with valid data make make slurm file:
for cmip in cmips:
    Clist = jc.readList(('../CMIP' + cmip + 'list'))
    Models = list(Clist.keys())
    for mm in Models:
        Model = Clist[mm]
        print(Model.name)
        if ((Model.Ogrid != 'Unstructured')):                
            for var in vars:
                for EXP in EXPs:
                    ENSs = Model.getENSs(var=var,EXP=EXP,vtype=vtype,gtype='*')
                    for ENS in ENSs:
                        gtype = 'gn'
                        # Check if files needed for computation exist:
                        Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)
                        if ((len(Files) == 0) & (cmip == '6')):
                            gtype = 'gr'
                            Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)
                            if ((len(Files) == 0) & (cmip == '6')):
                                gtype = 'gr1'
                                Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)

                        if len(Files) != 0:
                            # Check if computation has already been completed:
                            outfile = (Model.savedir + Model.name + '/' + EXP + '/' + ENS + '/' + var + '_' + Model.name + '_' + EXP + '_'
                                           + ENS + '_' + gtype + '_' + str(styr) + '01-' + str(fnyr) + '12_SC.nc')
                            
                            if not os.path.isfile(outfile):
                                Files.sort()
                                stf = int(Files[0].split('_')[-1].split('-')[0][:4])
                                fnf = int(Files[-1].split('_')[-1].split('-')[1][:4])
                                if ((stf <= styr) & (fnf >= fnyr)):
                                    print('computing: ' + outfile)
                                    # Generate submit script for lotus:
                                    lfile = ('slurm_files/SC_mean_' + var + '_' + Model.name + '_' +  EXP + '_' + ENS + '.slurm')

                                    line1 = '#!/bin/bash \n'
                                    line2 = '#SBATCH --account=short4hr \n'
                                    line3 = '#SBATCH -p short-serial-4hr \n'
                                    line4 = '#SBATCH -o %J.out \n'
                                    line5 = '#SBATCH -e %J.err \n'
                                    line6 = '#SBATCH --time 4:00:00 \n'
                                    line7 = '\n'
                                    line8 = 'conda activate working \n'
                                    line9 = ('python ../compute_means_SC.py ' + cmip + ' ' + Model.name + ' ' + EXP + ' ' + ENS + ' '
                                    + var + ' ' + vtype + ' ' + str(styr) + ' ' + str(fnyr) + ' ' + outfile + ' ' + gtype + ' ../' + lfile)

                                    # Out of memory models (second pass):
                                    #line2 = '#SBATCH -p high-mem \n'
                                    #line3 = '#SBATCH --mem=32000 \n'

                                    file = open(lfile,'w') 
                                    file.writelines([line1, line2, line3, line4, line5, line6, line7, line8, line9])

                                    file.close()



ACCESS1-0
ACCESS1-3
BNU-ESM
CCSM4
CESM1-BGC
CESM1-CAM5
CESM1-CAM5-1-FV2
CESM1-FASTCHEM
CESM1-WACCM
CFSv2-2011
CMCC-CESM
CMCC-CM
CMCC-CMS
CNRM-CM5
CNRM-CM5-2
CSIRO-Mk3-6-0
CanAM4
CanCM4
CanESM2
EC-EARTH
FGOALS-g2
FGOALS-gl
FGOALS-s2
FIO-ESM
GEOS-5
GFDL-CM2p1
GFDL-CM3
GFDL-ESM2G
GFDL-ESM2M
GFDL-HIRAM-C180
GFDL-HIRAM-C360
GISS-E2-H
GISS-E2-H-CC
GISS-E2-R
GISS-E2-R-CC
HadCM3
HadGEM2-A
HadGEM2-AO
HadGEM2-CC
HadGEM2-ES
IPSL-CM5A-LR
IPSL-CM5A-MR
IPSL-CM5B-LR
MIROC-ESM
MIROC-ESM-CHEM
MIROC4h
MIROC5
MPI-ESM-LR
MPI-ESM-MR
MPI-ESM-P
MRI-AGCM3-2H
MRI-AGCM3-2S
MRI-CGCM3
MRI-ESM1
NICAM-09
NorESM1-M
NorESM1-ME
bcc-csm1-1
bcc-csm1-1-m
inmcm4
ACCESS-CM2
ACCESS-ESM1-5
ACCESS-OM2
ACCESS-OM2-025
AWI-CM-1-1-HR
AWI-CM-1-1-LR
AWI-CM-1-1-MR
AWI-ESM-1-1-LR
BCC-CSM2-HR
BCC-CSM2-MR
BCC-ESM1
CAMS-CSM1-0
CAS-ESM2-0
CESM1-CAM5-SE-HR
CESM1-CAM5-SE-LR
CESM2
CESM2-FV2
CESM2-WACCM
CESM2-WACCM-FV2
CIESM
CMCC-CM2-HR4
CMCC-CM2-SR5
CMCC-CM2-VHR4
CMCC-ESM2
CNRM-CM6-1
CNRM-CM6-1-HR
CNRM-ESM2-1
CanESM5
CanESM5-CanOE
E3SM-1-0


In [ ]:
cmips = ['5','6']
EXPs = ['piControl']
vars  = ['thetao','uo','vo','so','thkcello','volcello','tauuo','tauvo','zos','msftbarot','hfds','wfo','hfy']
vtype = 'Omon'
nyr = 500

# Find all models with valid data make make slurm file:
for cmip in cmips:
    Clist = jc.readList(('../CMIP' + cmip + 'list'))
    Models = list(Clist.keys())
    for mm in Models:
        Model = Clist[mm]
        print(Model.name)
        if ((Model.Ogrid != 'Unstructured')):                
            for var in vars:
                for EXP in EXPs:
                    ENSs = Model.getENSs(var=var,EXP=EXP,vtype=vtype,gtype='*')
                    for ENS in ENSs:
                        gtype = 'gn'
                        # Check if files needed for computation exist:
                        Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)
                        if (((len(Files) == 0) & (cmip == '6')) | (Model.name == 'NorESM2-MM')):
                            gtype = 'gr'
                            Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)
                            if ((len(Files) == 0) & (cmip == '6')):
                                gtype = 'gr1'
                                Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)

                        if len(Files) != 0:
                            # Check if computation has already been completed:
                            outfile = (Model.savedir + Model.name + '/' + EXP + '/' + ENS + '/' + var + '_' + Model.name + '_' + EXP + '_'
                                           + ENS + '_' + gtype + '_first' + str(nyr) + '_SC.nc')
                            
                            if not os.path.isfile(outfile):
                                Files.sort()

                                print('computing: ' + outfile)
                                # Generate submit script for lotus:
                                lfile = ('slurm_files/SC_mean_first_' + var + '_' + Model.name + '_' +  EXP + '_' + ENS + '.slurm')

                                line1 = '#!/bin/bash \n'
                                line2 = '#SBATCH --account=short4hr \n'
                                line3 = '#SBATCH -p short-serial-4hr \n'
                                line4 = '#SBATCH -o %J.out \n'
                                line5 = '#SBATCH -e %J.err \n'
                                line6 = '#SBATCH --time 4:00:00 \n'
                                line7 = '\n'
                                line8 = 'conda activate working \n'
                                line9 = ('python ../compute_means_SC_first.py ' + cmip + ' ' + Model.name + ' ' + EXP + ' ' + ENS + ' '
                                + var + ' ' + vtype + ' ' + str(nyr) + ' ' + outfile + ' ' + gtype + ' ../' + lfile)

                                if ((Model.name == 'GFDL-CM4') | (Model.name == 'HadGEM3-GC31-MM') | (Model.name == 'MIROC4h')):
                                    # Out of memory models (second pass):
                                    line2 = '#SBATCH -p high-mem \n'
                                    line3 = '#SBATCH --mem=32000 \n'
                                    line6 = '#SBATCH --time 12:00:00 \n'

                                file = open(lfile,'w') 
                                file.writelines([line1, line2, line3, line4, line5, line6, line7, line8, line9])

                                file.close()